# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data['close'])
print(data['close'])

None
None


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     symbol_strings[i]

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A',
            ],
            index = my_columns
        ),
            ignore_index = True
        )        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,127.18,55.99,N/A
1,AAL,16.19,-1.17,N/A
2,AAP,170.38,24.7,N/A
3,AAPL,140.77,42.8,N/A
4,ABBV,111.89,25.03,N/A
...,...,...,...,...
500,YUM,112.30,31.95,N/A
501,ZBH,168.26,-223.96,N/A
502,ZBRA,428.16,48.25,N/A
503,ZION,48.15,20.68,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)


In [8]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,KMI,15.52,333.77,N/A
1,MAR,124.60,234.44,N/A
2,IQV,189.38,212.99,N/A
3,CMG,1516.12,181.47,N/A
4,REG,48.15,173.8,N/A
5,GLW,39.40,172.49,N/A
6,NOW,548.47,164.97,N/A
7,ADSK,302.56,163.34,N/A
8,DXCM,379.09,160.32,N/A
9,EL,257.29,157,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:1000000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = position_size / final_dataframe.loc[row, 'Price']
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,KMI,15.52,333.77,1.28866e+06
1,MAR,124.60,234.44,160514
2,IQV,189.38,212.99,105608
3,CMG,1516.12,181.47,13191.6
4,REG,48.15,173.8,415369
5,GLW,39.40,172.49,507614
6,NOW,548.47,164.97,36465.1
7,ADSK,302.56,163.34,66102.6
8,DXCM,379.09,160.32,52757.9
9,EL,257.29,157,77733.3


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# Price to earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
# Price to book ratio
pd_ratio = data[symbol]['advanced-stats']['priceToBook']
# Price to sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
editda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/editda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [13]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    


    for symbol in symbol_string.split(','):
        # Price to earnings ratio
        pe_ratio = data[symbol]['quote']['peRatio']
        # Price to book ratio
        pd_ratio = data[symbol]['advanced-stats']['priceToBook']
        # Price to sales ratio
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']

        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        editda = data[symbol]['advanced-stats']['EBITDA']
        
        # Enterprise Value divided by Gross Profit (EV/GP)
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value / editda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            en_to_ebitda = enterprise_value/gross_profit
        except TypeError:
            en_to_ebitda = np.NaN
            
        rv_dataframe = rv_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                en_to_ebitda,
                'N/A',
                'N/A',
            ],
            index = rv_columns
        ),
            ignore_index = True
        )        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.74,N/A,55.31,N/A,8.09,N/A,7.4,N/A,35.124821,N/A,14.026285,N/A,N/A
1,AAL,16.20,N/A,-1.14,N/A,-1.78,N/A,0.3978,N/A,-5.854028,N/A,1.395286,N/A,N/A
2,AAP,165.93,N/A,24.96,N/A,3.06,N/A,1.16,N/A,11.175351,N/A,2.574076,N/A,N/A
3,AAPL,143.66,N/A,42.8,N/A,37.16,N/A,8.58,N/A,30.341720,N/A,21.727682,N/A,N/A
4,ABBV,115.19,N/A,24.57,N/A,12.88,N/A,5.02,N/A,16.777704,N/A,10.227760,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.34,N/A,32.1,N/A,-4.1,N/A,5.8,N/A,23.038591,N/A,10.147183,N/A,N/A
501,ZBH,161.21,N/A,-223.38,N/A,2.89,N/A,4.72,N/A,27.554726,N/A,8.323942,N/A,N/A
502,ZBRA,427.78,N/A,48.31,N/A,11.34,N/A,5.23,N/A,32.326899,N/A,11.836468,N/A,N/A
503,ZION,50.11,N/A,20.6,N/A,1.15,N/A,2.69,N/A,11.575715,N/A,3.100975,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,211.25,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,243.30,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.87,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,33.12,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.38,N/A,14.79,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,54.90,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.59,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,162.09,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1946.48,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.38,N/A,31.89,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
# rv_dataframe = rv_dataframe[:50]
# rv_dataframe.reset_index(inplace = True)
# rv_dataframe.drop('index', axis=1, inplace = True)

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score( rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.74,N/A,55.31,86.9307,8.09,74.2574,7.4000,80.495,35.124821,85.1485,14.026285,81.1881,N/A
1,AAL,16.20,N/A,-1.14,15.4455,-1.78,4.75248,0.3978,2.57426,-5.854028,2.9703,1.395286,5.34653,N/A
2,AAP,165.93,N/A,24.96,49.3069,3.06,40.9901,1.1600,15.6436,11.175351,25.3465,2.574076,11.6832,N/A
3,AAPL,143.66,N/A,42.80,79.604,37.16,96.4356,8.5800,85.4455,30.341720,82.7723,21.727682,94.8515,N/A
4,ABBV,115.19,N/A,24.57,48.9109,12.88,85.1485,5.0200,70.297,16.777704,49.3069,10.227760,66.9307,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.34,N/A,32.10,65.1485,-4.10,4.35644,5.8000,74.1584,23.038591,69.703,10.147183,66.5347,N/A
501,ZBH,161.21,N/A,-223.38,2.57426,2.89,39.2079,4.7200,67.5248,27.554726,78.6139,8.323942,53.4653,N/A
502,ZBRA,427.78,N/A,48.31,82.1782,11.34,83.3663,5.2300,71.6832,32.326899,83.7624,11.836468,73.0693,N/A
503,ZION,50.11,N/A,20.60,42.6733,1.15,11.3861,2.6900,42.2772,11.575715,27.9208,3.100975,16.0396,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
#         pe_ratio = 
#         pd_ratio = 
#         ps_ratio = 
#         ev_to_ebitda = 
#         ev_to_gp = 
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.74,N/A,55.31,86.9307,8.09,74.2574,7.4000,80.495,35.124821,85.1485,14.026285,81.1881,81.604
1,AAL,16.20,N/A,-1.14,15.4455,-1.78,4.75248,0.3978,2.57426,-5.854028,2.9703,1.395286,5.34653,6.21782
2,AAP,165.93,N/A,24.96,49.3069,3.06,40.9901,1.1600,15.6436,11.175351,25.3465,2.574076,11.6832,28.5941
3,AAPL,143.66,N/A,42.80,79.604,37.16,96.4356,8.5800,85.4455,30.341720,82.7723,21.727682,94.8515,87.8218
4,ABBV,115.19,N/A,24.57,48.9109,12.88,85.1485,5.0200,70.297,16.777704,49.3069,10.227760,66.9307,64.1188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.34,N/A,32.10,65.1485,-4.10,4.35644,5.8000,74.1584,23.038591,69.703,10.147183,66.5347,55.9802
501,ZBH,161.21,N/A,-223.38,2.57426,2.89,39.2079,4.7200,67.5248,27.554726,78.6139,8.323942,53.4653,48.2772
502,ZBRA,427.78,N/A,48.31,82.1782,11.34,83.3663,5.2300,71.6832,32.326899,83.7624,11.836468,73.0693,78.8119
503,ZION,50.11,N/A,20.60,42.6733,1.15,11.3861,2.6900,42.2772,11.575715,27.9208,3.100975,16.0396,28.0594


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,41.92,N/A,-7.08,11.2871,0.5656,5.74257,0.7720,6.93069,4.528207,4.15842,0.766330,1.58416,5.94059
1,AAL,16.20,N/A,-1.14,15.4455,-1.7800,4.75248,0.3978,2.57426,-5.854028,2.9703,1.395286,5.34653,6.21782
2,UNM,26.50,N/A,5.53,19.604,0.4961,5.34653,0.4502,3.36634,2.902715,3.56436,0.441502,0.39604,6.45545
3,F,12.01,N/A,-290.93,2.37624,1.3600,17.0297,0.3529,1.58416,4.686610,4.35644,2.396816,11.0891,7.28713
4,MET,50.96,N/A,8.48,20.5941,0.6496,6.13861,0.7223,6.13861,4.830195,4.55446,0.697693,1.18812,7.72277
5,FTI,11.25,N/A,-0.90,15.6436,1.2000,12.8713,0.3881,2.17822,3.452538,3.76238,2.247716,10.6931,9.0297
6,BA,215.00,N/A,-25.99,7.52475,-9.8900,3.36634,1.9900,32.6733,-28.884465,1.58416,-198.177332,0.19802,9.06931
7,HIG,52.69,N/A,10.69,22.3762,1.0400,9.80198,0.8870,8.91089,4.447997,3.9604,0.911065,2.37624,9.48515
8,ALL,111.47,N/A,7.93,20,1.3700,17.3267,0.7863,7.52475,2.476796,3.36634,0.741662,1.38614,9.92079
9,ABC,109.13,N/A,-6.73,11.4851,-22.6000,1.9802,0.1171,0.39604,8.807048,13.2673,4.051845,22.5743,9.94059


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

Enter the value of your portfolio:1000000000


In [21]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = position_size / rv_dataframe.loc[row, 'Price']
    
rv_dataframe

/Users/Haruna/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,41.92,477099,-7.08,11.2871,0.5656,5.74257,0.7720,6.93069,4.528207,4.15842,0.766330,1.58416,5.94059
1,AAL,16.20,1.23457e+06,-1.14,15.4455,-1.7800,4.75248,0.3978,2.57426,-5.854028,2.9703,1.395286,5.34653,6.21782
2,UNM,26.50,754717,5.53,19.604,0.4961,5.34653,0.4502,3.36634,2.902715,3.56436,0.441502,0.39604,6.45545
3,F,12.01,1.66528e+06,-290.93,2.37624,1.3600,17.0297,0.3529,1.58416,4.686610,4.35644,2.396816,11.0891,7.28713
4,MET,50.96,392465,8.48,20.5941,0.6496,6.13861,0.7223,6.13861,4.830195,4.55446,0.697693,1.18812,7.72277
5,FTI,11.25,1.77778e+06,-0.90,15.6436,1.2000,12.8713,0.3881,2.17822,3.452538,3.76238,2.247716,10.6931,9.0297
6,BA,215.00,93023.3,-25.99,7.52475,-9.8900,3.36634,1.9900,32.6733,-28.884465,1.58416,-198.177332,0.19802,9.06931
7,HIG,52.69,379579,10.69,22.3762,1.0400,9.80198,0.8870,8.91089,4.447997,3.9604,0.911065,2.37624,9.48515
8,ALL,111.47,179420,7.93,20,1.3700,17.3267,0.7863,7.52475,2.476796,3.36634,0.741662,1.38614,9.92079
9,ABC,109.13,183268,-6.73,11.4851,-22.6000,1.9802,0.1171,0.39604,8.807048,13.2673,4.051845,22.5743,9.94059


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index=False)
writer.save()

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [30]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
float_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [33]:
column_formats = {
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['Price-to-Earnings Ratio', float_template],
    'E':['PE Percentile', percent_template],
    'F':['Price-to-Book Ratio', float_template],
    'G':['PB Percentile', percent_template],
    'H':['Price-to-Sales Ratio',float_template],
    'I':['PS Percentile', percent_template],
    'J':['EV/EBITDA', float_template],
    'K':['EV/EBITDA Percentile', percent_template],
    'L':['EV/GP', float_template],
    'M':['EV/GP Percentile', percent_template],
    'N':['RV Score', percent_template]
}

for column in column_format.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0],column_formats[column][1])


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [34]:
writer.save()